## 关注最下方阈值e的理解

In [1]:
import random
import os

In [2]:
# 全局变量
match_score = 6  # 匹配得分
dismatch_score = -3 #不匹配得分
gap_score = -2  # 插入空格得分
gap_character = '!' # 输出时代表插入空格


# 根据两个字符串计算得分矩阵
# 参数：两个字符串s1和s2
# 返回值：得分mat，几层列表的嵌套形式，s1第i个元素，s2第j个元素，对应mat[i][j]
#mat[i][j][0]: 数值，表示该位置最大得分值为多少，
#mat[i][j][1]: 列表，表示该最大得分值来自h_val,d_val,v_val的哪一个或者哪几个，即左方，左上角，上方
def compute_scores(s1, s2):
    # 初始化
    row = len(s1) + 1
    col = len(s2) + 1
    mat = [[[[None] for i in range(2)] for i in range(col)] for i in range(row)]
    # 第0行与第0列
    for i in range(row):
        mat[i][0] = [gap_score*i,[]]
    for j in range(col):
        mat[0][j] = [gap_score*j,[]]
    # 遍历两个字符串每个对应的位置
    for i in range(1,row):
        for j in range(1,col):
            score = match_score if (s1[i-1] == s2[j-1]) else dismatch_score
            # 三个取值来源，取最大值
            h_val = mat[i][j-1][0] + gap_score
            d_val = mat[i-1][j-1][0] + score
            v_val = mat[i-1][j][0] + gap_score
            o_val = [h_val, d_val, v_val]
            mat[i][j] = [max(o_val), [i+1 for i,v in enumerate(o_val) if v==max(o_val)]] # h = 1, d = 2, v = 3
    return mat


# 递归函数，在计算好的得分矩阵基础上，逆向寻找某位置最大得分值的路径
# 参数：得分矩阵/嵌套列表mat，当前两个字符串中的位置curr_i, curr_j，当前已有路径path
# 返回值：无，更新的是一个全局列表DIGIT_PATHES
def find_path(mat, curr_i, curr_j,  digit_paths, path=''):
    i = curr_i 
    j = curr_j 
    # 加上这一句判断，不用找到所有的对齐方式，否则序列教长时递归的复杂度太大
    # 找到最多4个即可结束递归
    if(len(digit_paths) >= 1):
        return
    if i == 0 and j==0:  # 两边同时结束，用2表示
        digit_paths.append(path)
        return 2
    dir_t = len(mat[i][j][1])  # matrix[i][j][1]表示该位置得分值的来源，可能有多个
    # 若dir_t小于等于1，则只有一个来源，不用考虑多种选择
    while dir_t<=1: 
        n_dir = mat[i][j][1][0] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0)) 
        path = path + str(n_dir)
        # 判断走三个方向的哪一个，1为左，2为左上，3为上，并更新当前位置
        if n_dir == 1: 
            j=j-1
        elif n_dir == 2:
            i=i-1
            j=j-1
        elif n_dir == 3:
            i=i-1
        # 更新方向取值
        dir_t = len(mat[i][j][1])
        if i == 0 and j==0:
            digit_paths.append(path)
            return 3
    # 若dir_t大于1，说明有多个方向来源，需要遍历，考虑每一个
    if dir_t>1:
        for dir_c in range(dir_t):
            n_dir = mat[i][j][1][dir_c] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0))
            tmp_path = path + str(n_dir)
            if n_dir == 1:
                n_i = i
                n_j = j-1
            elif n_dir == 2:
                n_i = i-1
                n_j = j-1
            elif n_dir == 3:
                n_i = i-1
                n_j = j
            find_path(mat, n_i, n_j, digit_paths, tmp_path)  # 递归调用


# 将数字方向表示的路径转化为原始字符串序列
# 参数：得分矩阵/嵌套列表mat，两个字符串s1和s2，不用输入DIGIT_PATHES，因为定义为global了
# 返回值：字符串对齐后的结果
def path_to_string(mat, s1, s2, digit_paths):
    i = len(s1)  #初始化矩阵行数 = 序列1长度+1
    j = len(s2)  # 初始化矩阵列数 = 序列2长度+1
    score = mat[i][j][0]  # 右下角得分值，即总分值
    l_i = i
    l_j = j
    answer = []  # 存储输出的序列
    find_path(mat, i, j, digit_paths)  # 调用函数
    # digit_paths 中记录的path是路径方向的数字编号，需要转化为字符串表示，存到answer中
    for elem in digit_paths:
        i = l_i-1
        j = l_j-1
        side_aln = ''
        top_aln = ''
        step = 0
        aln_info = []
        for n_dir_c in range(len(elem)):
            n_dir = elem[n_dir_c]
            score = mat[i+1][j+1][0]
            step = step + 1
            aln_info.append([step,score,n_dir])
            if n_dir == '2':
                side_aln = side_aln + s1[i]
                top_aln = top_aln + s2[j]
                i=i-1
                j=j-1
            elif n_dir == '1':
                side_aln = side_aln + gap_character
                top_aln = top_aln + s2[j]
                j=j-1
            elif n_dir == '3':
                side_aln = side_aln + s1[i]
                top_aln = top_aln + gap_character
                i=i-1
        answer.append([top_aln[::-1],side_aln[::-1]])
    return answer


def same_string(s1, s2):
    matrix = compute_scores(s1, s2)  # 计算得分
    digit_paths = []
    paths = path_to_string(matrix, s1, s2, digit_paths)  # 得到对齐的结果
    align1 = paths[0][0]  # 仅保留第一种对齐方式的两个字符串
    align2 = paths[0][1]
    # 遍历对齐结果，仅保留匹配上的部分
    i = 0
    same = ""
    while(i < len(align1)):
        if(align1[i] == align2[i]):  # 匹配的字符
            same += align1[i]
        #else:
        #    if tmp[-1] != "*":  # 为了间隔开不同的匹配段
        #        tmp += "...*"
        i+=1
    return [align1, align2, same]


In [3]:
# 定义函数，读取所有的输入txt文件，获得所有的报文数据
# 参数：文件夹名称，txt后缀
# 返回值：所有报文构成的列表，列表每个元素代表一条报文
def read_input(packet_name):
    
    packets = []  # 存放所有报文，每个元素代表一个报文
    file_list = os.listdir(packet_name)
    num = len(file_list)  # 文件夹下报文txt个数
    
    for i in range(1, num+1):
        
        # 每个文件名
        read_file_name = packet_name + '\\' + str(i) + '.txt'
    
        # 打开，整体读取，而非按行读取
        f = open(read_file_name, 'r')
        content = str(f.read())
        f.close()
        
        # 存入列表中
        packets.append(content)
    
    return packets

In [11]:
# 函数：对文件中同一种格式的报文进行多序列对比，得到固定字节组成的字符串
# 参数：文件夹名称，txt后缀，阈值e（决定保留或者删除两串的对齐结果，用于应对聚类错误的串）
# 返回值：固定字节组成的字符串

def format_string(packet_name, e):
    
    inputs = read_input(packet_name) 
    num = len(inputs)
    son = num // 10  # 10个报文作为一个子区间

    # 多次随机删除，防止聚类有误
    delete_ans = []  # 存放多次随即删除和阈值判断后的结果
    for i in range(5):
        random_inputs = random.sample(inputs, len(inputs)//2)  # 每次随机删除一半，即只保留一半的报文
        random_inputs.sort(key = lambda i:len(i))  # 按长度排序
        
        # 多次打乱，防止可能出现的匹配不完全
        shuffle_ans = []  # 存放多次随机打乱后的结果
        for j in range(5):
            shuffle_inputs = []
            for k in range(son):  # 将报文在各个小区间内打乱，random.shuffle会改变原有列表，而不是生成新列表
                tmp = random_inputs[k*10:(k+1)*10]
                random.shuffle(tmp)
                shuffle_inputs += tmp
            random_inputs = shuffle_inputs  # 作为下一次的输入，不是总对原始txt进行打乱
            
            # 初始化
            t = 0  # 记录第几轮遍历了
            raw = shuffle_inputs
            
            # 多轮遍历，直到只剩下一个格式串
            while len(raw)>1:
                same = []  # 保存本轮遍历提取的字符串
                num = len(raw)  # raw是本轮遍历的内容
                if(num%2 != 0):  # 防止奇数，i+1可能越界
                    num -= 1
                for k in range(0, num, 2):  # 遍历raw中串，进一步提取相同格式串
                    res = same_string(raw[k], raw[k+1])
                    if len(res[2]) > e:  # 阈值判断，只保留长度超过阈值的
                        same.append(res[2])
                raw = same  # 下一轮初始化
                t += 1
    
            if(len(same) != 0):  # 防止最后一轮遍历时由于不满足阈值限制导致结果为空
                shuffle_ans.append(same[0])  # 每次随机过程完全结束后，存储最终得到的格式串
            
        # 按频数排序，取频数最高的，作为本次随机打乱后的结果
        freq_dict = {}
        for k in shuffle_ans:
            freq_dict[k] = shuffle_ans.count(k)
        ordered = sorted(freq_dict.items(), key=lambda item:item[1], reverse=True)  # 按照字典的value排序
        print("\n第%d次随机删除后，多次随机打乱的结果按频率排序为：" % (i+1))
        print(ordered)
        print("频数最高的格式串为：", ordered[0][0])
        
        # 存储频数最高的格式串，作为最外面循环的一次结果
        delete_ans.append(ordered[0][0])
        print("当前存储的可能格式串为：", delete_ans)
    
    # 仍是频数最高的作为返回结果
    freq_dict = {}
    for k in delete_ans:
        freq_dict[k] = delete_ans.count(k)
    ordered = sorted(freq_dict.items(), key=lambda item:item[1], reverse=True)  # 按照字典的value排序
    
    return ordered[0][0]

In [12]:
# 函数：将固定字节组成的连续字符串拆分开，得到报文中对应的多个格式串
# 参数：文件名，字符串列表(只有一个字符串元素)
# 返回值：拆分后得到的字符串列表

def break_string(packet_name, e, same):
    
    # 读取报文，按照报文长度升序排序
    packets = read_input(packet_name)  
    packets.sort(key = lambda i:len(i))
    
    sample = random.sample(packets, 4)  # 随机选择4个报文
    res1 = same_string(sample[0], sample[1])
    res2 = same_string(sample[2], sample[3])
    res3 = same_string(res1[2], res2[2])
    
    if min(len(res1[2]), len(res2[2]), len(res3[2])) > e:  # 认为没有选到错误报文，开始拆分，下面所有代码都在if内
        all_res = []  # 记录与多个报文进行比对后的多个拆分结果
    
        # 得到根据不同报文得到的可能不同的拆分结果，记录在all_res中
        for i in sample:
            tmp1 = i  # 报文的字符串
            tmp2 = same  # 多序列对比提取的固定格式串
            start = 0
            end = 1
            res = ""  # 拆分结果用字符串表示
            while(end <= len(tmp2)):
                while tmp1.find(tmp2[start:end]) != -1:  # 匹配最长子串
                    end += 1
                    if(end == len(tmp2)+1):  # 最后一个字符属于某子串，防止end无穷加下去
                        break

                end -= 1
                res += tmp2[start:end]  # 输出该匹配的子串
                res += "####"  # 用####隔开不同格式串
                start = end  # 更新位置，为下一轮做准备
                if(end == len(tmp2)):
                    break
            res = res[:-4] # 去掉最后的####
            all_res.append(res)


        # 对all_res进行处理，得到最终的拆分结果，取频数最高的拆分结果
        freq_dict = {}
        for k in all_res:
            freq_dict[k] = all_res.count(k)
        ordered = sorted(freq_dict.items(), key=lambda item:item[1], reverse=True)  # 按照字典的value排序
        # print(ordered)
        # print("频数最高的拆分结果为：", ordered[0][0])

        return ordered[0][0]

### 在每个簇的文件夹上验证
随机删除次数由4改成1  
根据每个簇的报文长度，相应的设置阈值大小，此处尤其重要，因为不同类型仅有两个数字的差别

In [53]:
# cluster1
# 先多序列对比，得到固定格式串
print("第一步，进行多序列对比，得到固定格式字符串")
same_res = format_string("cluster1", 17)
print("\n最终得到的格式字符串为：\n", same_res)

第一步，进行多序列对比，得到固定格式字符串

第1次随机删除后，多次随机打乱的结果按频率排序为：
[('023619001d08ee307b', 4)]
频数最高的格式串为： 023619001d08ee307b
当前存储的可能格式串为： ['023619001d08ee307b']

第2次随机删除后，多次随机打乱的结果按频率排序为：
[('023619001d08ee307b', 5)]
频数最高的格式串为： 023619001d08ee307b
当前存储的可能格式串为： ['023619001d08ee307b', '023619001d08ee307b']

第3次随机删除后，多次随机打乱的结果按频率排序为：
[('023619001d08ee307b', 2)]
频数最高的格式串为： 023619001d08ee307b
当前存储的可能格式串为： ['023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b']

第4次随机删除后，多次随机打乱的结果按频率排序为：
[('023619001d08ee307b', 4)]
频数最高的格式串为： 023619001d08ee307b
当前存储的可能格式串为： ['023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b']

第5次随机删除后，多次随机打乱的结果按频率排序为：
[('023619001d08ee307b', 5)]
频数最高的格式串为： 023619001d08ee307b
当前存储的可能格式串为： ['023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b', '023619001d08ee307b']

最终得到的格式字符串为：
 023619001d08ee307b


In [54]:
# 拆分格式串
print("\n第二步，对固定格式字符串进行拆分")
break_res = break_string("cluster1", 17, same_res)
print("格式字符串拆分结果为：\n", break_res)


第二步，对固定格式字符串进行拆分
格式字符串拆分结果为：
 023619001d####08ee307b


In [55]:
# cluster2
# 先多序列对比，得到固定格式串
print("第一步，进行多序列对比，得到固定格式字符串")
same_res = format_string("cluster2", 17)
print("\n最终得到的格式字符串为：\n", same_res)

第一步，进行多序列对比，得到固定格式字符串

第1次随机删除后，多次随机打乱的结果按频率排序为：
[('02361900108ee307bc215', 1), ('02361900108ee307b6', 1), ('02361900108ee307bf', 1)]
频数最高的格式串为： 02361900108ee307bc215
当前存储的可能格式串为： ['02361900108ee307bc215']

第2次随机删除后，多次随机打乱的结果按频率排序为：
[('023619008108ee307b', 2)]
频数最高的格式串为： 023619008108ee307b
当前存储的可能格式串为： ['02361900108ee307bc215', '023619008108ee307b']

第3次随机删除后，多次随机打乱的结果按频率排序为：
[('0236190008ee307bd5', 1), ('02361900108ee307b3', 1), ('023619008108ee307bb9', 1)]
频数最高的格式串为： 0236190008ee307bd5
当前存储的可能格式串为： ['02361900108ee307bc215', '023619008108ee307b', '0236190008ee307bd5']

第4次随机删除后，多次随机打乱的结果按频率排序为：
[('023619008108ee307b9', 1), ('02361900108ee307b9a', 1)]
频数最高的格式串为： 023619008108ee307b9
当前存储的可能格式串为： ['02361900108ee307bc215', '023619008108ee307b', '0236190008ee307bd5', '023619008108ee307b9']

第5次随机删除后，多次随机打乱的结果按频率排序为：
[('023619008108ee307b', 2)]
频数最高的格式串为： 023619008108ee307b
当前存储的可能格式串为： ['02361900108ee307bc215', '023619008108ee307b', '0236190008ee307bd5', '023619008108ee307b9', '02361900810

In [56]:
# 拆分格式串
print("\n第二步，对固定格式字符串进行拆分")
break_res = break_string("cluster2", 17, same_res)
print("格式字符串拆分结果为：\n", break_res)


第二步，对固定格式字符串进行拆分
格式字符串拆分结果为：
 0236190081####08ee307b


In [57]:
# cluster3
# 先多序列对比，得到固定格式串
print("第一步，进行多序列对比，得到固定格式字符串")
same_res = format_string("cluster3", 17)
print("\n最终得到的格式字符串为：\n", same_res)

第一步，进行多序列对比，得到固定格式字符串

第1次随机删除后，多次随机打乱的结果按频率排序为：
[('023619002708ee307b', 5)]
频数最高的格式串为： 023619002708ee307b
当前存储的可能格式串为： ['023619002708ee307b']

第2次随机删除后，多次随机打乱的结果按频率排序为：
[('023619002708ee307b', 1)]
频数最高的格式串为： 023619002708ee307b
当前存储的可能格式串为： ['023619002708ee307b', '023619002708ee307b']

第3次随机删除后，多次随机打乱的结果按频率排序为：
[('023619002708ee307b', 1), ('023619002708ee307ba', 1)]
频数最高的格式串为： 023619002708ee307b
当前存储的可能格式串为： ['023619002708ee307b', '023619002708ee307b', '023619002708ee307b']

第4次随机删除后，多次随机打乱的结果按频率排序为：
[('023619002708ee307b', 1)]
频数最高的格式串为： 023619002708ee307b
当前存储的可能格式串为： ['023619002708ee307b', '023619002708ee307b', '023619002708ee307b', '023619002708ee307b']

第5次随机删除后，多次随机打乱的结果按频率排序为：
[('023619002708ee307b', 2)]
频数最高的格式串为： 023619002708ee307b
当前存储的可能格式串为： ['023619002708ee307b', '023619002708ee307b', '023619002708ee307b', '023619002708ee307b', '023619002708ee307b']

最终得到的格式字符串为：
 023619002708ee307b


In [58]:
# 拆分格式串
print("\n第二步，对固定格式字符串进行拆分")
break_res = break_string("cluster3", 17, same_res)
print("格式字符串拆分结果为：\n", break_res)


第二步，对固定格式字符串进行拆分
格式字符串拆分结果为：
 0236190027####08ee307b


In [59]:
# cluster4
# 先多序列对比，得到固定格式串
print("第一步，进行多序列对比，得到固定格式字符串")
same_res = format_string("cluster4", 17)
print("\n最终得到的格式字符串为：\n", same_res)

第一步，进行多序列对比，得到固定格式字符串

第1次随机删除后，多次随机打乱的结果按频率排序为：
[('023619003c08ee307b', 5)]
频数最高的格式串为： 023619003c08ee307b
当前存储的可能格式串为： ['023619003c08ee307b']

第2次随机删除后，多次随机打乱的结果按频率排序为：
[('023619003c08ee307b', 5)]
频数最高的格式串为： 023619003c08ee307b
当前存储的可能格式串为： ['023619003c08ee307b', '023619003c08ee307b']

第3次随机删除后，多次随机打乱的结果按频率排序为：
[('023619003c08ee307b', 5)]
频数最高的格式串为： 023619003c08ee307b
当前存储的可能格式串为： ['023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b']

第4次随机删除后，多次随机打乱的结果按频率排序为：
[('023619003c08ee307b', 5)]
频数最高的格式串为： 023619003c08ee307b
当前存储的可能格式串为： ['023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b']

第5次随机删除后，多次随机打乱的结果按频率排序为：
[('023619003c08ee307b', 3)]
频数最高的格式串为： 023619003c08ee307b
当前存储的可能格式串为： ['023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b', '023619003c08ee307b']

最终得到的格式字符串为：
 023619003c08ee307b


In [60]:
# 拆分格式串
print("\n第二步，对固定格式字符串进行拆分")
break_res = break_string("cluster4", 17, same_res)
print("格式字符串拆分结果为：\n", break_res)


第二步，对固定格式字符串进行拆分
格式字符串拆分结果为：
 023619003c####08ee307b
